In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

/home/gitonga/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train = pd.read_csv('titanic_train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
nulls = pd.DataFrame ({'Count' : train.count()})

nulls['Nulls'] = nulls['Count'].apply(lambda x: train.shape[0] - x)
nulls['Nulls %'] = nulls['Count'].apply(lambda x: 100 * ((train.shape[0] - x) / train.shape[0]))


nulls = nulls[nulls['Nulls %'] != 0].sort_values(by = 'Nulls').reset_index()

nulls.columns.values[0] = 'Column'
nulls

,Column,Count,Nulls,Nulls %
0,Embarked,889,2,0.224467
1,Age,714,177,19.865320
2,Cabin,204,687,77.104377


In [3]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

In [4]:
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)
train.drop('Cabin',axis=1,inplace=True)
train.dropna(inplace=True)
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [5]:
sex = pd.get_dummies(train['Sex'],drop_first=True)
embark = pd.get_dummies(train['Embarked'],drop_first=True)

In [6]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
train = pd.concat([train,sex,embark],axis=1)
train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,1,0,3,22.0,1,0,7.2500,1,0,1
1,2,1,1,38.0,1,0,71.2833,0,0,0
2,3,1,3,26.0,0,0,7.9250,0,0,1
3,4,1,1,35.0,1,0,53.1000,0,0,1
4,5,0,3,35.0,0,0,8.0500,1,0,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train.drop('Survived',axis=1), 
                                                    train['Survived'], test_size=0.30, 
                                                    random_state=101)

### Logistic Regression

In [8]:
logmodel = LogisticRegression(C = 1.0)
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
predictions = logmodel.predict(X_test)

In [10]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.81      0.93      0.86       163
          1       0.85      0.65      0.74       104

avg / total       0.82      0.82      0.81       267



In [11]:
confusion_matrix(y_test,predictions)

array([[151,  12],
       [ 36,  68]])

### Decision Tree

In [12]:
dtmodel = DecisionTreeClassifier()
dtmodel.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [13]:
predictions = dtmodel.predict(X_test)

In [14]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.80      0.80      0.80       163
          1       0.68      0.68      0.68       104

avg / total       0.75      0.75      0.75       267



In [15]:
confusion_matrix(y_test,predictions)

array([[130,  33],
       [ 33,  71]])

### Random Forest

In [16]:
rfmodel = RandomForestClassifier()
rfmodel.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
predictions = rfmodel.predict(X_test)

In [18]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.83      0.91      0.87       163
          1       0.83      0.70      0.76       104

avg / total       0.83      0.83      0.82       267



In [19]:
confusion_matrix(y_test,predictions)

array([[148,  15],
       [ 31,  73]])

### KNN

In [20]:
knmodel = KNeighborsClassifier()
knmodel.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [21]:
predictions = knmodel.predict(X_test)

In [22]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.66      0.80      0.73       163
          1       0.54      0.36      0.43       104

avg / total       0.61      0.63      0.61       267



In [23]:
confusion_matrix(y_test,predictions)

array([[131,  32],
       [ 67,  37]])

### Gradient Boosting Classifier

In [24]:
gbmodel = GradientBoostingClassifier()
gbmodel.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [25]:
predictions = gbmodel.predict(X_test)

In [26]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.84      0.90      0.87       163
          1       0.83      0.74      0.78       104

avg / total       0.84      0.84      0.84       267



In [27]:
confusion_matrix(y_test,predictions)

array([[147,  16],
       [ 27,  77]])

### SVC

In [31]:
svcmodel = SVC()
svcmodel.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [32]:
predictions = svcmodel.predict(X_test)

In [33]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.61      0.99      0.76       163
          1       0.00      0.00      0.00       104

avg / total       0.37      0.61      0.46       267



In [34]:
confusion_matrix(y_test,predictions)

array([[162,   1],
       [104,   0]])